# Import a Molecules/Atom from cartesian coordinates:
**Task:** 
1. Create a Molecule object from its cartesian coordinates
2. Ask for its atomic and molecular properties

## [Water molecule](https://en.wikipedia.org/wiki/Properties_of_water) ($H_{2}O$):

<img 
src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/H2O_2D_labelled.svg/2560px-H2O_2D_labelled.svg.png"
alt="water" 
width="200" 
height="100" 
/>

### Properties:
- Number of molecules: 1
- Number of atoms: 3
- Atomic symbols: O, H, H
- Atomic masses [g/mol]: 
    - O: 15.999
    - H: 1.008
    - H: 1.008
- Molar mass [g/mol]: 18.015
- Cartesian coordinates [Angstrom]:
    ```XYZ
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000
    ```
- Bond distance [Angstrom]:
    - H-O: 0.9584 (95.84 pm)
    - H-H: 1.5151
- Bond angle [Degrees]:
    - H-O-H: 104.45
    - H-H-O: 37.78

In [4]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
# importing de Molecule Class
from src.base_molecule import Molecule

In [54]:
# creating a Molecular object. You can use a list, a dictionary (the key MUST be "atoms")
# or another Molecule object (see below)

water=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]
water_dict = {"atoms": [("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]}

water_molecule = Molecule(water_dict)
water_molecule = Molecule(water)

In [59]:
import numpy as np
from scipy.optimize import shgo
from pyscf import gto, scf
import py3Dmol


## en éste bloque vemos la molécula inicial que ponemos a trabajar con el shgo


def arming_fragments(geo):
    water_molecule = Molecule(geo)
    water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)
    water_molecule_2 = water_molecule.add_fragments(geo)
    return water_molecule_2 


bi_water = arming_fragments(water_dict)

xyz_view = py3Dmol.view(width=300,height=200)
xyz_view.addModel(str(bi_water),'xyz')
xyz_view.setStyle({'stick':{}})

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [61]:
import numpy as np
from scipy.optimize import shgo
from pyscf import gto, scf

def optimization(x):
    """Funcion a optimizar. Primero se introduce la molécula de agua (water_dict), 
    luego, con la clase Molecule la trasladamos una cierta distancia (x=y=z=1.5) y luego
    agregamos el fragmento water_dict, para tener dos moléculas de agua separadas en 1.5 angs en todas direcciones. 

    Luego, a la molécula de 2 aguas, la trasladamos y rotamos en función de x, un 1-D array.
    A esa molécula, le calculanos la energía, siguiendo la base por defecto y RHF.  
    Parameters
    ----------
    x : [1-D Array, de 6 variables, en este caso]
        [variables x,y,z de traslación y los siguientes 3 de rotación]

    Returns
    -------
    []
        [Retorna la energía total del sistema, dado los parámetros en x]
    """
    water_dict = {"atoms": [("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]}

    water_molecule = Molecule(water_dict)
    water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)


    water_molecule_2 = water_molecule.add_fragments(water_dict)

    water_molecule_2 = water_molecule_2.translate(0,x=x[0],y=x[1],z=x[2])
    water_molecule_2 = water_molecule_2.rotate(0,x[3],x[4],x[5])
    
    with open('water.xyz', 'w') as f:
        f.write(str(water_molecule_2))
    
    mol = gto.Mole()
    mol.atom = "water.xyz"
    mol.build()

    mf = scf.RHF(mol)
    mf.kernel()


    return mf.energy_tot()

#x = np.array([0.0,0.0,0.0,0,0,0])

#water = optimization(x)

#print(water) 
bounds = [(-3,3),(-3,3),(-3,3),(0,180),(0,180),(0,180)]
#bounds = [(None, None), ]*6
#aquí habría que probar con el método sobol, y también con otros métodos de optimización. 
minimos = shgo(optimization, bounds=bounds, sampling_method='simplicial')

print(minimos.xl, minimos.funl)


converged SCF energy = -149.857752687531
converged SCF energy = -149.860385449484
converged SCF energy = -149.859886470921
converged SCF energy = -149.86051453889
converged SCF energy = -149.860385449484
converged SCF energy = -149.860361127314
converged SCF energy = -149.860361127314
converged SCF energy = -149.860385449484
converged SCF energy = -149.860385449484
converged SCF energy = -149.860361127314
converged SCF energy = -149.860361127314
converged SCF energy = -149.860272545331
converged SCF energy = -149.860272545331
converged SCF energy = -149.86051453889
converged SCF energy = -149.86051453889
converged SCF energy = -149.860514538891
converged SCF energy = -149.860272545331
converged SCF energy = -149.860272545331
converged SCF energy = -149.860136723112
converged SCF energy = -149.860679352995
converged SCF energy = -149.860679352995
converged SCF energy = -149.860136723112
converged SCF energy = -149.860136723112
converged SCF energy = -149.860136723112
converged SCF energ

In [72]:
import py3Dmol
def graficator(x):
    """Funcion para observar las geometrías optimizadas con el método shgo.

    Parameters
    ----------
    x : 1-D array, 6 coordinates]
        [aquí se debe poner los parámetros optimizados]

    Returns
    -------
    []
        [molécula]
    """
    water_dict = {"atoms": [("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]}

    water_molecule = Molecule(water_dict)
    water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)


    water_molecule_2 = water_molecule.add_fragments(water_dict)

    water_molecule_2 = water_molecule_2.translate(0,x=x[0],y=x[1],z=x[2])
    water_molecule_2 = water_molecule_2.rotate(0,x[3],x[4],x[5])
    
    return water_molecule_2

#aquí fijamos cuál mínimo vamos a utilizar
x = minimos.xl[2]
print(x)
molecule = graficator(x)
print(type(molecule))
xyz_view = py3Dmol.view(width=300,height=200)
xyz_view.addModel(str(molecule),'xyz')
xyz_view.setStyle({'stick':{}})

[-2.51409777e+00 -2.52411262e+00  7.98962677e-01  1.79997485e+02
  3.22877658e-02  1.40928486e-02]
<class 'src.base_molecule.Molecule'>


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# Atomic Simulation Environment
# https://wiki.fysik.dtu.dk/ase/index.html
# !pip install --upgrade --user ase


# ChemML
# https://hachmannlab.github.io/chemml/index.html
# !pip install chemml

In [ ]:
# NGLview
# https://github.com/nglviewer/nglview
# !pip install nglview

# ---------------------------------------
# pytraj 
# https://amber-md.github.io/pytraj/latest/index.html
# !pip install pytraj
